# In this tutorial, we discuss sparse vector techniques.


**Sparse Vector Technique** (SVT) is one of the most fundamental algorithmic tools in differential privacy that
allows the algorithm to screen potentially an unbounded number of adaptively chosen queries while paying a cost of privacy
only for a small number of queries that passes a predefined threshold.

In this tutorial, we revisit the classic SVT (based on Laplace noise) and recent new variants of SVT algorithms (e.g., SVT with Gaussian noise) from
[Zhu et.al., NeurIPS-20](https://proceedings.neurips.cc/paper/2020/hash/e9bf14a419d77534105016f5ec122d62-Abstract.html)


In SVT, the input is a stream of adaptively chosen queries $q_1, ..., q_k, ...$. The queries are provided with a sequence of thresholds
$T_1, ..., T_k, ...$. The goal of SVT is to release a binary vector $(\perp, \top)^k$ at every time $k$,
$\top$ indicates that the correponding query answers $q_i(D)$ is above the threshold $T_i$ and $\perp$ indivates below.
To release this vector differential privately, the classic SVT first perturbs the threshold with a Laplace noise
$\rho$. Then each individual query $q_i(D)$ is perturbed with another Laplace noise $\nu_i$ before comparing against the
perturbed threshold $T_i + \rho$ to determine the binary decision, until the stopping condition --- the c-th $\top$ arrives.


In [22]:
from autodp.mechanism_zoo import ExactGaussianMechanism, PureDP_Mechanism
from autodp.transformer_zoo import Composition
import matplotlib.pyplot as plt
%matplotlib inline 

## Let's first define the classic SVT.

We define a Laplace-based SVT with c=1.

In [23]:
from autodp.mechanism_zoo import GaussianSVT_Mechanism, LaplaceSVT_Mechanism
import numpy as np

# b denotes the Laplace noise scale of \rho (\nu = 2\rho by default). k is the maximum length before the algorithm stop. k could
# be infinite in the classic SVT. When k is not required, we provide an improved RDP bound.
#
params = {'b':1,'k':100, 'c':1}
lap_svt = LaplaceSVT_Mechanism(params)
delta = 1e-5
eps = lap_svt.get_eps(delta)

/usr/local/lib/python3.8/site-packages/scipy/optimize/_optimize.py:2884: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom
/Users/zhuyuqing/github_proj/YQ-autoDP/autodp/autodp/rdp_bank.py:363: RuntimeWarning: overflow encountered in sinh
  cdp_bound = np.sinh(alpha * tilde_eps) - np.sinh((alpha - 1) * tilde_eps)
/Users/zhuyuqing/github_proj/YQ-autoDP/autodp/autodp/rdp_bank.py:363: RuntimeWarning: invalid value encountered in double_scalars
  cdp_bound = np.sinh(alpha * tilde_eps) - np.sinh((alpha - 1) * tilde_eps)


##  We next provide a Gaussian-noise based SVT.

Gaussian SVT adds Gaussian noise to perturb both the threshold and the query.

In [24]:
# setting rdp_c_1=True implies we use RDP-based Gaussian-SVT with c=1. sigma_nu is the noise scale added to each query and
# sigma is the noise scale added to the threshold.
params = {'rdp_c_1':True, 'sigma':1, 'sigma_nu':1, 'k':100}
gau_svt = GaussianSVT_Mechanism(params)
delta = 1e-5
eps = gau_svt.get_eps(delta)

## How can we compare two SVT variants?

Given a predetermined privacy budget $(\epsilon, \delta)$ and the cut-off c, we compare the length each
SVT-like algorithm can screen before stopping.

We estimate the length of answered queries with Negative Binomial Distribution. For example, in the case of Gaussian-SVT,
with the threshold noise $z \sim N(0, \sigma_1^2)$ and the query noise $\nu \sim N(0,\sigma_2^2)$, denote K is
the number of queries answered when hits c. Notie that $K|\rho=z$ follows a Negative Binomial
Distribution, $E[K|z]$ can be estimated with $\frac{cF_\nu(T+z)}{1-F_\nu(T+z)}$, where
$F_\nu$ is the CDF of the noise $\nu$ and queries are all zeros.

Let's consider setting the cut-off parameter $c=20$, the threshold T=700 for all queries and $\delta=1e-10$.

In [25]:
# calibrate noise with a given privacy budget.
from autodp.calibrator_zoo import eps_delta_calibrator,generalized_eps_delta_calibrator
from autodp.mechanism_zoo import LaplaceSVT_Mechanism, GaussianSVT_Mechanism

eps = 0.1
general_calibrate = generalized_eps_delta_calibrator()
params_lap = {'b':None,'k':100, 'c':20}
lap_svt_mech = general_calibrate(LaplaceSVT_Mechanism, eps, delta, [0, 1000], params=params, para_name='b', name='lap_SVT')
print(lap_svt_mech.name, lap_svt_mech.params, lap_svt_mech.get_approxDP(delta))


print(f'in algorithm f eps,delta = ({eps},{delta}) ==> Noise level b=', params['b'])
lambda_rho = params['b']
ret_number = []
lambda_nu = 2*lambda_rho # this is the default choice of lambda_nu used in SVT.
repeat_n = 10000
margin = 700 # the threshold
c = 20
# estimate the empirical mean of the length of the answered queries.
for i in range(repeat_n):
    rho = np.random.laplace(loc=0, scale=lambda_rho)
    fail_prob = 0.5 * np.exp(-(margin + rho) / lambda_nu)
    expect_steps = c * (1 - fail_prob) / fail_prob
    ret_number.append(expect_steps)
ret_laplace = sum(ret_number) * 1.0 / repeat_n



KeyError: 'c'